In [14]:
library(ggplot2)
library(dplyr)
library(readr)
library(tidyr)
library(lmPerm)

### Kiểm định

In [3]:
df <- read_csv('data/clean_outliers.csv')

Rows: 253680 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (22): Diabetes_012, HighBP, HighChol, CholCheck, BMI, Smoker, Stroke, He...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
df <- df %>% distinct()
sum(duplicated(df))

[1] 0

In [6]:
# kiểm định chi-square với monte carlo simulation
chisq_test <- function(df, var1, var2) {
  observed <- table(df[[var1]], df[[var2]])
  chisq <- chisq.test(observed, simulate.p.value = TRUE, B = 5000)
  return(chisq)
}

In [8]:
# kiểm định kruskal-wallis
kruskal_test <- function(df, var1, var2) {
  kruskal <- kruskal.test(df[[var1]] ~ df[[var2]], data = df)
  return(kruskal)
}

#### Biến giải thích với predictor

In [13]:
vars <- c('HighBP', 'HighChol', 'CholCheck', 'Stroke', 'HeartDiseaseorAttack', 
        'PhysActivity', 'DiffWalk', 'GenHlth', 'Age', 'Education', 'Income')

chi_square_results <- data.frame(variable = character(), p_value = numeric(), reject = logical(), stringsAsFactors = FALSE)

for (var in vars) {
    chi_square <- chisq_test(df, 'Diabetes_012', var)
    reject <- chi_square$p.value < 0.05
    chi_square_results <- rbind(chi_square_results, data.frame(variable = var, p_value = chi_square$p.value, reject = reject))
}

chi_square_results


variable,p_value,reject
<chr>,<dbl>,<lgl>
HighBP,0.00019996,TRUE
HighChol,0.00019996,TRUE
CholCheck,0.00019996,TRUE
Stroke,0.00019996,TRUE
HeartDiseaseorAttack,0.00019996,TRUE
PhysActivity,0.00019996,TRUE
DiffWalk,0.00019996,TRUE
GenHlth,0.00019996,TRUE
Age,0.00019996,TRUE


In [11]:
chisq_test(df, 'Diabetes_012', 'AnyHealthcare')


	Pearson's Chi-squared test

data:  observed
X-squared = 146.18, df = 2, p-value < 2.2e-16


In [35]:
vars_quanlity <- c('BMI', 'PhysHlth', 'MentHlth')

for (var in vars_quanlity) {
   results <- aov(formula=df[[var]] ~ df[['Diabetes_012']], data = df)
   print(summary(results))
}


                         Df Sum Sq Mean Sq F value Pr(>F)    
df[["Diabetes_012"]]      1   9709    9709   10816 <2e-16 ***
Residuals            229769 206254       1                   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
                         Df  Sum Sq Mean Sq F value Pr(>F)    
df[["Diabetes_012"]]      1   53820   53820    6075 <2e-16 ***
Residuals            229769 2035632       9                   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
                         Df  Sum Sq Mean Sq F value Pr(>F)    
df[["Diabetes_012"]]      1   11380   11380   767.6 <2e-16 ***
Residuals            229769 3406656      15                   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [39]:
for (var in vars_quanlity) {
   results <- kruskal_test(df, var, 'Diabetes_012')
   print(results)
}


	Kruskal-Wallis rank sum test

data:  df[[var1]] by df[[var2]]
Kruskal-Wallis chi-squared = 11827, df = 2, p-value < 2.2e-16


	Kruskal-Wallis rank sum test

data:  df[[var1]] by df[[var2]]
Kruskal-Wallis chi-squared = 4574.4, df = 2, p-value < 2.2e-16


	Kruskal-Wallis rank sum test

data:  df[[var1]] by df[[var2]]
Kruskal-Wallis chi-squared = 125.54, df = 2, p-value < 2.2e-16



#### Biến giải thích với nhau